# Tensorflowを使ってみる

まずは線形回帰で

$$y = 0.1 * x  + 0.3$$ 

を学習することをやってみる。

まずは、ランダムに`x_data`を生成して、そこから上記の数式に値をいれた`y_data`を取得する

In [1]:
import tensorflow as tf
import numpy as np

x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3
print("x_data -> ", x_data)
print("y_data -> ", y_data)

x_data ->  [ 0.87822241  0.03435802  0.61389858  0.72873425  0.11537459  0.60941231
  0.20453164  0.25644848  0.52539408  0.51763147  0.02325199  0.20357937
  0.68771172  0.26826584  0.80503482  0.71996832  0.18834728  0.19419245
  0.61154366  0.50840551  0.27119976  0.16651374  0.12257904  0.51482487
  0.99325681  0.24660948  0.16741131  0.39474511  0.22038484  0.52723044
  0.82761902  0.07838525  0.17879717  0.73014289  0.08789682  0.25970617
  0.18870877  0.35354593  0.5095849   0.71133196  0.52399832  0.21330319
  0.07023059  0.61943442  0.68295532  0.92720121  0.61127126  0.20195648
  0.40215907  0.06200561  0.74029875  0.51204956  0.21873741  0.33181921
  0.44420293  0.809461    0.0322804   0.65216804  0.16343296  0.58795637
  0.31044978  0.26988786  0.993164    0.25087088  0.29007167  0.12441614
  0.00408988  0.43185651  0.77115953  0.48946148  0.64036083  0.46374351
  0.41974771  0.43258893  0.09363202  0.98393965  0.73826015  0.29927459
  0.98338354  0.47268099  0.7875219   0.

次に、学習するモデルを

$$ y = W * x  + b $$

とおいて、`W`と`b`を変数とする。そのため、tensorflowのVariableとして定義する。

In [2]:
W = tf.Variable(tf.random_uniform([1], -1, 1))
b = tf.Variable(tf.zeros([1]))
print(W,b)
y = W * x_data + b
print(y)

<tensorflow.python.ops.variables.Variable object at 0x10d822940> <tensorflow.python.ops.variables.Variable object at 0x10d41c2b0>
Tensor("add:0", shape=TensorShape([Dimension(100)]), dtype=float32)


`print`するとtensoflowのVariableとして定義されていることがわかる。また、`y`はTenorのインスタンスで、`"add:0"`という表現を持っていることがわかる。

さて、学習するために、コスト関数を定義する。またその後、コスト関数を最小化するための手法として最急降下法のインスタンスを、学習率0.5で生成し、コスト関数を最小化する。

In [3]:
loss = tf.reduce_mean(tf.square(y - y_data))
print("loss->", loss)
optimizer = tf.train.GradientDescentOptimizer(0.5)
print("optimizer->", optimizer)
train = optimizer.minimize(loss)
print("train->", train)

loss-> Tensor("Mean:0", shape=TensorShape([]), dtype=float32)
optimizer-> <tensorflow.python.training.gradient_descent.GradientDescentOptimizer object at 0x10e10f128>
train-> name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"



すべての値を初期化する。この場合、初期化されるのは、`W`と`b`。

In [4]:
init = tf.initialize_all_variables()

tensorflowを実行するときには、Sessionを生成する

In [5]:
sess = tf.Session()
sess.run(init)

sessionはrunすることにより、Tensorのインスタンスを実行したり、Variableを表示したりすることができる。

In [6]:
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

0 [ 0.74136662] [-0.01180235]
20 [ 0.27561066] [ 0.21652415]
40 [ 0.1478828] [ 0.27723911]
60 [ 0.11305594] [ 0.29379392]
80 [ 0.1035599] [ 0.29830784]
100 [ 0.10097065] [ 0.29953861]
120 [ 0.10026467] [ 0.29987422]
140

 [ 0.10007217] [ 0.29996571]
160 [ 0.1000197] [ 0.29999065]
180 [ 0.10000539] [ 0.29999745]
200 [ 0.10000148] [ 0.2999993]
